# Chapter 6 - Lattices and Cryptography

Lattice Based Cryptography provides a public key cryptosystem that has "quantum resistance" (ie there is no quantum algorithm to solve it quickly) as well as faster encrytion and decryption.

This project aims to explore the LLL and BKZ algorithms used in Lattice Based Cryptography using SAGE.

## 6.1 - A congruential public key cryptosystem

In [77]:
# imports
import random
import time

### Key creation

In [78]:
def congruential_key_creation(bits):
    """
    Chooses pseudorandom integers q, f, g.
    
    Computes public key (q, h)
    
    Returns private and public keys: [(f, g), (q, h)]
    
    NOTE: needs a number of bits that is not too small
    for example 10 is enough (if smaller it might get stuck).
    Since encryption usually requires 128 to 256 bits of security,
    it would not be feasible to use less than 10 bits as this is insecure.
    
    BITS must be large for the encryption function not to get stuck.
    """
    # choosing a large positive integer q
    q = random.getrandbits(bits)   # gives an integers with "bits" random bits
    
    FieldQ = Zmod(q)


    # choosing the other 2 secret positive integers
    a, b, c = 0, sqrt(q/4), sqrt(q/2)   # 
    
    while True:
        f = random.randint(a, floor(c))          # the function only takes integers as inputs so must round (down)
        g = random.randint(floor(b), floor(c))   # the values of b and c (down because we want a strict inequality
                                                 # and randint has a weak one)
        if gcd(f, g) == 1:
            try:
                f = FieldQ(f)
                g = FieldQ(g)
                f_inverse = f^(-1)
                break
            except ZeroDivisionError:
                continue
    
    # compute h
    h = mod((f_inverse*g),q)
    
    return [(f, g), (FieldQ, q, h)]

### Encryption

In [79]:
def congruential_encryption(public_key_tuple, message):
    """
    Encrypts the message using the public key
    
    NOTE: only works for string with at most 12 characters - otherwise it will get stuck.
    """
    FieldQ, q, h = public_key_tuple
    
    # translate from string to integer using a base
    a, b, c = 0, sqrt(q/4), sqrt(q/2)
    
    while True:
        # choose a base randomly
        base = random.randint(33, 36)   # why this range?
        m = Integer(message, base=base)   # 0 < m < sqrt(q/4)
        
        if (0 < m) and (m < b):
            break
    
    # r will be random (ephemeral key)
    r = random.randint(a, floor(c))   # 0 < r < sqrt(q/2)
    
    # compute ciphertext
    e = mod((r*h + m), q)   # 0 < e < q
    
    # return the ciphertext and the base to convert back to string
    return (e, base)
    
    

### Decryption

In [80]:
def congruential_decryption(private_key_tuple, public_key_tuple, message_tuple):
    """
    Decrypts the message using the private key.
    """
    
    f, g = private_key_tuple
    FieldQ, q, h = public_key_tuple
    e, base = message_tuple
    
    # compute part 1 of decryption
    # for this part it is mod q
    a = mod((f*e), q)   # 0 < a < q
    
    # since the integers f, g, r, m are small due to restrictions,
    # a = rg + fm without the need of the mod q
    
    
    # compute part 2 of the decryption
    # for this part it is mod g
    # thus we need to modify a, f and g so that it is all mod g
    g_in_g = Integer(g)
    FieldG = Zmod(g_in_g)
    
    f_in_g = Integer(f)
    f_in_g = FieldG(f_in_g)
    f_inverse_g = f_in_g^(-1)
    
    a_in_g = Integer(a)
    
    b = mod((f_inverse_g*a_in_g), g_in_g)   # 0 < b < g
    
    # convert integer back to text
    plaintext = Integer(b, base=base).str(base=base)
    
    # return plaintext
    return plaintext
    

### Example

We generate the public and private keys. Then, we encrypt it using the public keys. Finally, we decrypt it with the private keys. The original message is returned (in lowercase and without any spaces).

In [81]:
# key creation
pr, pb = congruential_key_creation(128)

# encryption
cptx = congruential_encryption(pb, 'Quaternion')  # maximum of 12 characters

# decryption
pltx = congruential_decryption(pr, pb, cptx)

# original message
print(pltx)

quaternion


### Attack

To attack the algorithm you can brute force it. However, it is very likely that if you find $F$ and $G$ such that $$Fh \equiv G \pmod q \quad \text{and} \quad F = \mathcal{O}(\sqrt(q)) \quad  \text{and} \quad G = \mathcal{O}(\sqrt(q)).$$ 

Which is
$$Fh = G +qR.$$

In other words, to get the vector $(F, G)$ we can solve the following equation:
$$F(1, h) - R(0, q) = (F, G).$$

So to solve it you want to find a short vector in the set
$$L = \{a_1v_1 + a_2v_2 \,\,|\,\, a_1, a_2 \in \mathbb{Z}\},$$
and the vector has lenght $\mathcal{O}(\sqrt(q))$.

This is a two-dimensional lattice problem.

In [82]:
def congruential_attack(public_key_tuple, message_tuple):
    """
    Attacks the congruential public key cryptosystem to try
    and eavesdrop.
    """
    
    FieldQ, q, h = public_key_tuple
    e, base = message_tuple
    
    
    
    return hacked_plaintext

## 6.2 - Subset-sum problems and knapsack cryptosystems

In [83]:
def subset_sum_superincreasing(list_M, S):
    """
    Computes the solution vector y for the subset_sum problem.
    """
    
    n = len(list_M)
    
    y = zero_vector(ZZ, n)
    
    for i in reversed(range(0, n)):
        if S >= list_M[i]:
            y[i] = 1
            S = S - list_M[i]
    
    return y

### Example

In [84]:
M = [3, 11, 24, 50, 115]
S = 142

print(subset_sum_superincreasing(M, S))

(1, 0, 1, 0, 1)


### Key creation

In [104]:
def superincreasing_seq(length):
    """
    Makes a superincreasing sequence.
    
    This function is a simple way of creating such a pseudorandom sequence
    of superincreasing integers, for the sake of applying the algorithm.
    """
    seq = zero_vector(length)
    
    seq[0] = random.randint(2**n, 7*(2**n))   # first value is a random integer between 0 and 10
    
    for i in range(1, length):
        seq[i] = random.randint(2*seq[i-1] + 1, 7*seq[i-1])   # makes the next value superincreasing
    
    return seq

def knapsack_key_creation(n, bits):
    """
    Creates the private and public keys of the subset sum encryption.
    
    BITS must be large for the loop not to get stuck.
    """
    r_super_seq = superincreasing_seq(n)
    
    while True:
        B = random.getrandbits(bits)
        if B > 2*r_super_seq[n-1]:
            break
    
    while True:
        A = random.getrandbits(bits)
        try:
            FieldB = Zmod(B)
            A = FieldB(A)
            A_inverse = A^(-1)
            break
        except ZeroDivisionError:
            continue
    
    A = Integer(A)
    
    M = zero_vector(n)
    
    for i in range(n):
        M[i] = mod(A*r_super_seq[i], B)   # 0 <= Mi < B
    
    return (r_super_seq, A, B), (M, n)

In [105]:
print(superincreasing_seq(4))

(13858, 66013, 393616, 1920077)


### Encryption

In [106]:
def knapsack_encryption(public_key_tuple, bin_plaintext):
    """
    Encrypts a binary message.
    
    NOTE make sure the bin_plaintext doesn't have the 0b at the beggining.
    
    FORMATS for bin_plaintext allowed are binary string or list.
    """
    M, n = public_key_tuple
    
    bin_plain_list = [int(bin_plaintext[i]) for i in range(n)]
    
    bin_plain_vec = vector(bin_plain_list)
    
    S = bin_plain_vec.dot_product(M)
    
    # return ciphertext
    return S

### Decryption

In [107]:
def knapsack_decryption(private_key_tuple, S):
    """
    Decrypts the binary message.
    """
    
    r_super_seq, A, B = private_key_tuple
    
    FieldB = Zmod(B)
    
    A = FieldB(A)
    
    A_inverse = A^(-1)
    
    S_prime = mod(A_inverse*S, B)
    
    bin_plaintext = subset_sum_superincreasing(r_super_seq, S_prime)
    
    # return binary string
    return bin_plaintext

### Example with randomly generated binary string

In [109]:
# create public and private keys
pr, pb = knapsack_key_creation(12, 128)

# unpack public key to obtain dimensions of binary message
M, n = pb

# create random binary message (accepted string or list) - for the purpose of the example
bin_mess = zero_vector(n)
for i in range(n):
    if random.random() <= 0.71:
        bin_mess[i] = 1

print(f"Secret binary message: {bin_mess}")
        
# encrypt message
S = knapsack_encryption(pb, bin_mess)

# decrypt message
print(f"Decrypted message: {knapsack_decryption(pr, S)}")

Secret binary message: (1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0)
Decrypted message: (1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0)


## 6.3 - A brief review of vector spaces

In [127]:
def gram_schmidt_orthogonal(basis_list):
    """
    Given a basis of vectors for vector space V (list),
    the function returns an ORTHOGONAL ONLY basis for V (list).
    
    NOTE the vectors may not be normalized.
    """
    orthogonal_list = []
    n = len(basis_list)
    
    # leave the first basis vector (row 0) as the same
    orthogonal_list.append(basis_list[0])
    
    # loop through the others until orthogonal
    for i in range(1, n):
        new_orth_vec = basis_list[i]

        for j in range(i):
            
            mu_ij = basis_list[i].dot_product(orthogonal_list[j]) / (orthogonal_list[j].norm()**2)
            
            new_orth_vec = new_orth_vec - mu_ij*orthogonal_list[j]
            
            orthogonal_list.append(new_orth_vec)
            
            #mu_ij = zero_vector(n)
            #mu_ij[j] = basis_mat.row(i).dot_product(orthogonal_basis.row(j)) / (orthogonal_basis.row(j).norm()**2)
            
            #orthogonal_basis.set_row(orthogonal_basis.row(i), (orthogonal_basis.row(i) - mu_ij.dot_product(orthogonal_basis.row(j))))
    
    return orthogonal_list

print(gram_schmidt_orthogonal([vector([1,0]),vector([1,1])]))

[(1, 0), (0, 1)]


## 6.4 - Lattices: Basic definitions and properties

## 6.5 - Short vectors in lattices

## 6.6 - Babai's algorithm and using a "good" basis to solve apprCVP

In [20]:
def babai_closest_vertex(basis_list, w):
    """
    This algorithm solves the closest vector problem (CVP).
    
    NOTE this only works when we havea good basis for the lattice. That is, the basis
    vectors are reasonable orthogonal to one another.
    """
    basis_mat = matrix([vector for vector in basis_list])
    
    t_values = basis_mat.solve_left(w)
    
    print(type(t_values))
    

    
babai_closest_vertex([vector([1,0]), vector([1, 2])], vector([0,1]))

<class 'sage.modules.vector_rational_dense.Vector_rational_dense'>


## 6.7 - Cryptosystems based on hard lattice problems

## 6.8 - The GGH public key cryptosystem

## 6.9 - Convolution polynomial rings

## 6.10 - The NTRU public key cryptosystem

## 6.11 - NTRU as a lattice cryptosystem

## 6.12 - Lattice reduction algorithms

## 6.13 - Applications of LLL to cryptanalysis

## Constructing a basis for a q-lattice

In [ ]:
# parameters
m, n, q = 5, 3, 101

# compute RREF
A = random_matrix(GF(q), n, m)
A.echelonize()

# stack A on top of a matrix accounting for modular reductions
N = A.change_ring(ZZ)
S = matrix(ZZ, m-n, n).augment(q * identity_matrix(m-n))
N.stack(S, subdivide= True)

In [ ]:
# other lattice
sage.crypto.gen_lattice(m=10, seed=42, type="modular")

## The LLL algorithm

In [ ]:
sage.crypto.gen_lattice(m=10, seed=42, type="modular")
A.LLL(delta=0.99, eta=0.51)

## The BKZ algorithm

In [ ]:
A = sage.crypto.gen_lattice(m=100, seed=42, q=next_prime(2^20))
B = A.BKZ(block_size=60, proof=False)
B[0].norm().log(2).n()

## References

Hoffstein, J., Pipher, J., Silverman, J. H., & Silverman, J. H. (2008). An introduction to mathematical cryptography (Vol. 1). New York: springer.